# Ridge regression

## by ANDRIANARIVONY Henintsoa Salohy

In [60]:
import numpy as np
import scipy
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.linear_model import Ridge 

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = datasets.load_diabetes()
X_train, y_train = data.data, data.target

In [61]:
import math

In [62]:
def fit_inverse(X, y, alpha=1.0, fit_intercept=False):
    """Direct method using the inverse"""
    if fit_intercept:
        ones = np.ones((len(X), 1))
        X = np.hstack((X, ones))
    # I matrix identity
    I = np.eye(len(X[0]), len(X[0]))
    if fit_intercept:
        I[len(X[0]) - 1][len(X[0]) - 1] = 0.0
    # Xh is the transpose of X
    Xh = np.transpose(X) 
    return np.linalg.inv(Xh @ X + (alpha * I)) @ Xh @ y

In [63]:
w = fit_inverse(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
assert error <= 1e-11
print("prediction error: ", error)

prediction error:  2.581619852266006e-14


In [107]:
def fit_qr_linear_regression(X, y):
    """ QR approach for linear regression from previous assignement"""
    # Xh is the transpose of X
    Xh = np.transpose(X)
    Q, R = np.linalg.qr(Xh @ X)
    w1 = np.transpose(Q) @ Xh @ y
    return scipy.linalg.solve_triangular(R, w1)

def fit_qr(X, y, alpha=1.0, fit_intercept=False):
    """QR approach"""
    if fit_intercept:
        ones = np.ones((len(X), 1))
        X = np.hstack((X, ones))
    # I matrix identity
    I = np.eye(len(X[0]), len(X[0]))
    if fit_intercept:
        I[len(X[0]) - 1][len(X[0]) - 1] = 0.0
    X = np.append(X, math.sqrt(alpha) * I, axis=0)
    y = np.append(y, np.zeros(len(X[0])))
    return fit_qr_linear_regression(X, y)

In [65]:
w = fit_qr(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
#assert error <= 1e-11
print("prediction error: ", error)

prediction error:  2.8024162869992768e-14


In [102]:
def fit_svd(X, y, alpha=1.0, fit_intercept=False):
    """SVD approach"""
    if fit_intercept:
        ones = np.ones((len(X), 1))
        X = np.hstack((X, ones))
    # I matrix identity
    I = np.eye(len(X[0]), len(X[0]))
    if fit_intercept:
        I[0][0] = 0.
    U, s, Vh = np.linalg.svd(X, full_matrices=False)
    Uh = np.transpose(U)
    V = np.transpose(Vh)
    S = np.diag(s)
    term1 = S @ S + alpha * I
    for i in range(len(S)):
        if term1[i][i] != 0:
            term1[i][i] = 1 / term1[i][i]
    term2 = S @ Uh @ y
    return V @ term1 @ term2

In [103]:
w = fit_svd(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
#assert error <= 1e-11
print("prediction error: ", error)

prediction error:  6.929611182397926e-14


# Everything in a class

In [69]:
class RidgeRegression():
    def __init__(self, fit_intercept=True, method="inverse", alpha=1.0):
        self.w = 0
        self.fit_intercept = fit_intercept # bias
        self.method = method
        self.alpha = alpha
    
    def fit(self, X, y):
        if self.method == "inverse":
            self.w = fit_inverse(X, y, alpha=self.alpha, fit_intercept=self.fit_intercept)
        elif self.method == "qr":
            self.w = fit_qr(X, y, alpha=self.alpha, fit_intercept=self.fit_intercept)
        else:
            self.w = fit_svd(X, y, alpha=self.alpha, fit_intercept=self.fit_intercept)
        
    def predict(self, X):
        if self.fit_intercept:
            ones = np.ones((len(X), 1))
            X = np.hstack((X, ones))
        return X @ self.w

## without bias

In [70]:
# OTHER APPROACHES
sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = RidgeRegression(fit_intercept=False, method="inverse", alpha=0.1)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
assert error <= 1e-11
print("prediction error inverse: ", error)

model_qr = RidgeRegression(fit_intercept=False, method="qr", alpha=0.1)
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
assert error_qr <= 1e-11
print("prediction error qr: ", error_qr)

model_svd = RidgeRegression(fit_intercept=False, method="svd", alpha=0.1)
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
assert error_svd <= 1e-11
print("prediction error svd: ", error_svd)

prediction error inverse:  8.662459094449379e-14
prediction error qr:  9.899953250799413e-14
prediction error svd:  4.949976625398971e-14


## with bias

In [108]:
# OTHER APPROACHES
sk_model = Ridge(fit_intercept=True, alpha=0.1)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = RidgeRegression(fit_intercept=True, method="inverse", alpha=0.1)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
assert error <= 1e-11
print("prediction error inverse: ", error)

model_qr = RidgeRegression(fit_intercept=True, method="qr", alpha=0.1)
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
#assert error_qr <= 1e-11
print("prediction error qr: ", error_qr)

model_svd = RidgeRegression(fit_intercept=True, method="svd", alpha=0.1)
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
assert error_svd <= 1e-11
print("prediction error svd: ", error_svd)

prediction error inverse:  1.0093912749924575e-15
prediction error qr:  7.183930383655086e-16
prediction error svd:  2.384008211280126e-14
